# Create Images of Sprites

Load in the sprite data

In [1]:
import re
sprites_files = ["../iridisalpha/src/graphics/sprites.asm",
                 "../iridisalpha/src/graphics/bonusphase_sprites.asm",
                 "../iridisalpha/src/graphics/enemy_sprites.asm"]
sprites_data = {}
for sprites_file in sprites_files:
    input_file = open(sprites_file,'r')
    sprite_data = []
    for l in input_file.readlines():
        if "SPRITE" in l:
            if sprite_data:
                sprites_data[sprite_name] = sprite_data
            sprite_name = l[18:20].strip().lower()
            sprite_data = []
            continue

        m = re.findall(r"[0-1]{24}",l)
        if not m:
            continue
        bits = m[0]
        sprite_line = []
        for i in range(0,23,2):
            bitpair = bits[i:i+2]
            sprite_line += [bitpair]
            sprite_line += [bitpair]
        sprite_data += [sprite_line]
    if sprite_data:
        sprites_data[sprite_name] = sprite_data


Get the color data from the sprite movements file

In [2]:
import gzip
log_file = "IridisAlphaTitleEnemySpriteMovements.txt.gz"
input_file = gzip.open(log_file,'rt')

movements = [ [] for _ in range(8) ]
for l in input_file.readlines():
    # the x and y co-ords for each sprite
    if "C:d000" in l:
        s = l[9:59]
        s = s.replace(' ',',').replace(',,',',')
        s = s.split(',')
        co_ords = [(int(s[i],16), int(s[i+1],16)) for i in range(0,len(s),2)]
        continue

    # sprite_colors
    if "C:d025" in l:
        s = l[9:40]
        s = s.replace(' ',',').replace(',,',',')
        sprite_colors_array = s.split(',')
        multi_color0 = sprite_colors_array[0]
        multi_color1 = sprite_colors_array[1]
        sprite_colors = sprite_colors_array[2:]
        continue

    # the most significant bit for each sprite's x co-ord
    if "C:d010" in l:
        msb = int(l[9:11],16)
        continue

    # The current sprite value
    if "C:07f8" in l:
        s = l[9:33]
        s = s.replace(' ',',').replace(',,',',')
        sprite_values = s.split(',')
        continue
        
    # current planet
    # change back to 78b0 in unmodified rom
    if "C:78b3" in l:
        planet = int(l[9:11],16)
        continue

    # current level
    # 49c6 in unmodified rom
    if not "C:49c9" in l:
        continue
    s = l[9:33]
    s = s.replace(' ',',').replace(',,',',')
    levels = s.split(',')
    level = int(levels[planet],16)

    # Add the most significant bit to each sprite's x value
    for i,v in enumerate([1,2,4,8,16,32,64,128]):
        if msb & v:
            x,y = co_ords[i]
            co_ords[i] = (x+256,y)

    for i, spr in enumerate(sprite_values):
        movements[i] += [(sprite_values[i], co_ords[i], (planet,level),
                         (multi_color0,multi_color1,sprite_colors[i]))]


Function for actually drawing the sprite

In [3]:
from PIL import Image, ImageColor

def paintSprite(origin, img, sprite, colors):
    multicol0, multicol1,color = colors
    colormap = {
        "01": multicol0,
        "10": color,
        "11": multicol1,
    }
    
    pixels = img.load()
    x_o, y_o = origin
    if sprite not in sprites_data:
        return
    bit_array = sprites_data[sprite]
    for y, l in enumerate(bit_array):
        for x,bit in enumerate(l):
            if bit == "00":
                continue
            pixel_color = ImageColor.getrgb(fhex_to_rgb[colormap[bit]])
            pixels[x_o+x, y_o+y] = pixel_color
    return img


In [52]:
surfaces = {
    0: Image.open("../src/planets/planet1Surface.png"),
    1: Image.open("../src/planets/planet2Surface.png"),
    2: Image.open("../src/planets/planet3Surface.png"),
    3: Image.open("../src/planets/planet4Surface.png"),
    4: Image.open("../src/planets/planet5Surface.png"),
}
#img = Image.open(file_name)
#background = Image.new('RGB', (504,312), "black")
#background.paste(img, (60,20))

for k, surface in surfaces.items():
    tmpimg = surfaces[k]
    tmpimg = tmpimg.crop((100,0,155,tmpimg.height))
    img = Image.new( 'RGB', (35,40), "black")
    img.paste(tmpimg, (0,23))
    surfaces[k] = img
#surfaces[0].resize((bkg.width * 10, bkg.height * 10), Image.NEAREST)


Write out the sprites to image files

In [68]:
import os
from collections import defaultdict
from sprite_map import sprite_map 
from colors_map import *

bits=8
width = 24
height = 21

for j, mvt in enumerate(movements):
    for i,(sprite, _origin, (_planet,_level), colors) in enumerate(mvt):
        if sprite not in sprite_map:
            continue
        sprite_name = sprite_map[sprite]
        #fname = f'sprites-gallery/sprite_{sprite_name}_{"".join(colors)}.png'
        fname = f'sprites-gallery/sprite_{int(sprite,16)}.png'
        if os.path.isfile(fname):
            continue
        img = Image.new( 'RGBA', (width,height))
        origin = (0,0)
        img = paintSprite(origin, img, sprite, colors)
        bkg = surfaces[_planet]
        bkg.paste(img, (5,5))
        bkg = bkg.resize((bkg.width * 10, bkg.height * 10), Image.NEAREST)
        bkg.save(fname)

for sprite in sprites_data:
    print(sprite)
    fname = f'sprites-gallery/sprite_{int(sprite,16)}.png'
    if os.path.isfile(fname):
        continue
    print(f"Doing {sprite}")
    img = Image.new( 'RGBA', (width,height))
    origin = (0,0)
    img = paintSprite(origin, img, sprite, colors)
    bkg = surfaces[_planet]
    bkg.paste(img, (5,5))
    bkg = bkg.resize((bkg.width * 10, bkg.height * 10), Image.NEAREST)
    bkg.save(fname)



a0
a1
a2
a3
a4
a5
a6
a7
a8
a9
aa
ab
ac
ad
ae
af
b0
b1
b2
b3
b4
b5
b6
b7
b8
b9
ba
bb
bc
bd
be
bf
c0
c1
c2
c3
c4
c5
c6
c7
c8
c9
ca
cb
cc
cd
ce
cf
d0
d1
d2
d3
d4
d5
d6
d7
d8
d9
da
db
dc
Doing dc
dd
Doing dd
de
df
Doing df
e0
e1
Doing e1
e2
Doing e2
e3
e4
e5
e6
e7
e8
e9
ea
eb
ec
ed
ee
ef
f0
f1
Doing f1
f2
Doing f2
f3
Doing f3
f4
f5
Doing f5
f6
f7
f8
f9
fa
fb
fc
fd
fe
ff
00
Doing 00
01
Doing 01
02
Doing 02
03
Doing 03
04
Doing 04
05
Doing 05
06
Doing 06
07
Doing 07
08
Doing 08
09
Doing 09
0a
Doing 0a
0b
Doing 0b
21
22
23
24
25
26
27
28
29
2a
2b
2c
2d
2e
2f
30
31
32
33
34
35
36
37
38
39
3a
3b
3c
3d
3e
3f


In [67]:
sprite_data[1]

['00',
 '00',
 '00',
 '00',
 '00',
 '00',
 '11',
 '11',
 '00',
 '00',
 '00',
 '00',
 '00',
 '00',
 '00',
 '00',
 '11',
 '11',
 '00',
 '00',
 '00',
 '00',
 '00',
 '00']

In [63]:
for sprite in sprite_map:
    if not sprite:
        continue
    print(f"\\newrow{{{int(sprite,16)}}}{{{sprite_map[sprite]}}}")

\newrow{247}{LICKERSHIP}
\newrow{250}{LICKERSHIP_INV}
\newrow{33}{BALLOON}
\newrow{34}{BALLOON}
\newrow{35}{BIRD}
\newrow{36}{BIRD}
\newrow{37}{BIRD}
\newrow{38}{BIRD}
\newrow{39}{SMALL_BALL}
\newrow{40}{SMALL_BALL}
\newrow{41}{SMALL_BALL}
\newrow{42}{QBERT_SQUARES}
\newrow{43}{QBERT}
\newrow{44}{LOZENGE}
\newrow{45}{MAGIC_MUSHROOM}
\newrow{46}{INV_MAGIC_MUSHROOM}
\newrow{47}{SMALLBALL_AGAIN}
\newrow{48}{CAMEL}
\newrow{49}{FLYING_DART}
\newrow{50}{FLYING_DART}
\newrow{51}{FLYING_DART}
\newrow{52}{FLYING_DART}
\newrow{53}{WINGED_BALL}
\newrow{54}{WINGED_BALL}
\newrow{55}{WINGED_BALL}
\newrow{56}{MONEY_BAG}
\newrow{57}{MONEY_BAG}
\newrow{58}{TITAN_ROCKET}
\newrow{63}{BUBBLE}
\newrow{59}{BOUNCY_RING}
\newrow{60}{BOUNCY_RING}
\newrow{61}{BOUNCY_RING}
\newrow{62}{BUBBLE}
\newrow{160}{FLYING_SAUCER}
\newrow{161}{FLYING_SAUCER}
\newrow{162}{FLYING_SAUCER}
\newrow{163}{FLYING_COMMA}
\newrow{164}{FLYING_COMMA}
\newrow{165}{FLYING_DOT}
\newrow{166}{FLYING_DOT}
\newrow{167}{BOLAS}
\newrow{168}{BO